# Problema de transporte discreto

Dadas dos distribuciones de probabilidad discretas:

* $X$: con soporte $x_1<x_2<\ldots<x_m$ con probabilidades $P(X=x_i) = p_i$.
* $Y$: con soporte $y_1<y_2<\ldots<y_n$ con probabilidades $P(Y=y_j) = q_j$.

Queremos hallar una distribución conjunta $A=(a_{ij}) \geqslant 0$ tal que:

$$\min_{a_{ij}} \sum_{i,j} a_{ij} |x_i-y_j|$$

sujeto a:

$$\sum_j a_{ij} = p_i \quad \forall i=1,\ldots,m,$$

$$\sum_i a_{ij} = q_j \quad \forall j=1,\ldots,n.$$

In [1]:
## Para instalar los paquetes del Project.toml la primera vez
#using Pkg; Pkg.instantiate()

In [2]:
using JuMP, GLPK, Ipopt

#m=3
x=[1.0;2.0;3.0]
p=[0.2;0.4;0.4]

#n=2
y=[0;4.0]
q=[0.5;0.5]

#model = JuMP.Model(GLPK.Optimizer)
model = JuMP.Model(Ipopt.Optimizer)


m=length(x)
n=length(y)

#Calculo los pesos |xi-yj|
W=abs.(x*ones(1,n) - ones(m,1)*y')

@variable(model,A[1:m,1:n]>=0)

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ))

model

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: A

In [3]:
optimize!(model)

LoadError: UndefVarError: libipopt not defined

In [4]:
objective_value(model)

LoadError: OptimizeNotCalled()

In [5]:
#Extraigo el valor optimo
value.(A)

LoadError: OptimizeNotCalled()

In [6]:
#Extraigo los multiplicadores de p
lambda = dual.(conp)

LoadError: OptimizeNotCalled()

In [7]:
#Extraigo los multiplicadores de q
mu = dual.(conq)

LoadError: OptimizeNotCalled()

In [8]:
W

3×2 Matrix{Float64}:
 1.0  3.0
 2.0  2.0
 3.0  1.0

In [9]:
lambda * ones(1,n) + ones(m,1)*mu

LoadError: UndefVarError: lambda not defined

# Problema de transporte discreto - Cantidad de destinos variable

Dadas dos distribuciones de probabilidad discretas:

* $X$: con soporte $x_1<x_2<\ldots<x_m$ con probabilidades $P(X=x_i) = p_i$.
* $Y$: con soporte $y_1<y_2<\ldots<y_n$ con probabilidades $P(Y=y_j) = q_j$.

Queremos hallar una distribución conjunta $A=(a_{ij}) \geqslant 0$ tal que:

$$\min_{a_{ij}} \sum_{i,j} a_{ij} |x_i-y_j| + \epsilon \sum_{j} \phi(q_{j})$$

sujeto a:

$$\sum_j a_{ij} = p_i \quad \forall i=1,\ldots,m,$$

$$\sum_i a_{ij} = q_j \quad (VAR).$$

## Caso 1: 

Consideramos $\phi(x) = \frac{x^{2}}{2}$

In [10]:
using JuMP, GLPK, Ipopt
#revisar paquete distances.JL

#m=3
x=10*rand(10) #[1.0;2.0;3.0;4;5;6;7;8;9;10] #--> Trabajar con elementos de R^2
p=ones(10)

#n=2
y=[0;7;15]

#epsilon
eps = 0 #¿Qué pongo?

model = JuMP.Model(GLPK.Optimizer)
#model = JuMP.Model(Ipopt.Optimizer)


m=length(x)
n=length(y)

#Calculo los pesos |xi-yj|
W=abs.(x*ones(1,n) - ones(m,1)*y')

@variable(model,A[1:m,1:n]>=0)
@variable(model,q[1:n])

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ))# + eps/2 * (q'*q) )

model

A JuMP Model
Minimization problem with:
Variables: 33
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 13 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 30 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: A, q

In [11]:
optimize!(model)

In [12]:
objective_value(model)

21.16096183911447

In [13]:
#Extraigo el valor optimo
value.(A)

10×3 Matrix{Float64}:
 0.0  1.0  0.0
 0.0  1.0  0.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [14]:
value.(q)

3-element Vector{Float64}:
 4.0
 6.0
 0.0

In [15]:
#Extraigo los multiplicadores de p
lambda = dual.(conp)

10×1 Matrix{Float64}:
 0.6245198880530829
 3.3197663803474127
 1.7169916387741158
 2.686493183107783
 2.1421984048827527
 2.282045503497301
 2.9218142484726806
 0.39198635762129364
 2.8241159907870217
 2.251030243571032

In [16]:
#hacerlo en R^2,dentro de un cuadrado de 0,0 a 1,1
#plotear con el comando scatter los x, con diferentes colores los que tienen el 1 en la primer columna, los que tienen el 1 en la segunda col, etc.
#manera más sencilla con marker.z
#spliteo el X y corro la optimización, voy cambiando los cargadores
#

## Caso 2: Trabajando en $R^{2}$

Consideramos $\phi(x) = \frac{x^{2}}{2}$

In [17]:
#m=los que quiera
m = 5000
x = sortslices(rand(m,2), dims = 1);
p = ones(m);

#n=4
#y=[[1/3 1/3]; [1/3 2/3]; [2/3 1/3]; [2/3 2/3]];
#y=[[1/3 1/3]; [2/3 2/3]];
y = sortslices(rand(6,2), dims = 1);
n=size(y)[1];

In [18]:
using JuMP, GLPK, Ipopt
#revisar paquete distances.JL

#epsilon
eps = 0#0.001 #200

#model = JuMP.Model(GLPK.Optimizer)
model = JuMP.Model(Ipopt.Optimizer)

#Calculo los pesos |xi-yj|
w1 = (x[:,1]*ones(1,n) - ones(m,1)*y[:,1]').^2
w2 = (x[:,2]*ones(1,n) - ones(m,1)*y[:,2]').^2
W=(w1+w2).^(1/2)

@variable(model,A[1:m,1:n]>=0)
@variable(model,q[1:n])#<=833)

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ) + eps/2 * (q'*q) )

model

A JuMP Model
Minimization problem with:
Variables: 30006
Objective function type: QuadExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5006 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 30000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: A, q

In [19]:
optimize!(model);

LoadError: UndefVarError: libipopt not defined

In [20]:
objective_value(model)

LoadError: OptimizeNotCalled()

In [21]:
#Extraigo el valor optimo
value.(A)

LoadError: OptimizeNotCalled()

In [22]:
value.(q)

LoadError: OptimizeNotCalled()

In [23]:
#Extraigo los multiplicadores de p
lambda = dual.(conp);

LoadError: OptimizeNotCalled()

In [24]:
using VoronoiCells
using GeometryBasics
using Plots
using Random

#Redondea los valores de A para que sean 0 o 1 (Probar)
Ar = round.(value.(A));

gr()
Plots.GRBackend()

#Separa los puntos por destino
X = [x[findall(x->x==1, Ar[:,i]),1] for i in 1:n]
Y = [x[findall(x->x==1, Ar[:,i]),2] for i in 1:n]
points = [y[:,1], y[:,2]]

#Conjuntos de Voronoi
rect = Rectangle(Point2(0, 0), Point2(1, 1))
tess = voronoicells(points, rect);

scatter(X, Y, aspect_ratio=:equal) #Grafica los origenes, con color por destino
scatter!(points, markersize = 10, label = "Destinos") #Plotea los destinos
annotate!([(points[n][1] + 0.02, points[n][2] + 0.03, Plots.text(n)) for n in 1:10])
plot!(tess, legend = :topleft)

LoadError: ArgumentError: Package VoronoiCells not found in current path:
- Run `import Pkg; Pkg.add("VoronoiCells")` to install the VoronoiCells package.
